![](https://raw.githubusercontent.com/wandb/wandb/508982e50e82c54cbf0dd464a9959fee0e1740ad/.github/wb-logo-lightbg.png)
<!--- @wandbcode{dataval-course-01} -->

In [ ]:
from dataval.dataset import WeatherDataset
from dataval.train import CatBoostTrainer

import os
import matplotlib.pyplot as plt
import pandas as pd
import wandb
from wandb.integration.catboost import WandbCallback

os.environ["WANDB_QUIET"] = "true" # Let's keep the output clean

# Let's start a new W&B run to track our work
run = wandb.init(project="ml-dataval-tutorial")

# Training Pipelines

In this notebook, we load the weather dataset (described in https://arxiv.org/pdf/2107.07455.pdf). We split the dataset into weekly partitions. Then we create 2 pipelines: one that trains a model on the first week & deploys on all following weeks, and another that continually trains & deploys on each consecutive pair of weeks.

In [ ]:
# Load dataset

ds = WeatherDataset(os.path.join(os.getcwd(), "canonical-paritioned-dataset"), sample_frac=0.2)

## Train-Once

We use a catboost model, optimizing for RMSE of weather prediction. We use off-the-shelf models/parameters (no ensembling or uncertainty estimation to keep things simple).

In [ ]:
first_iter = True
test_mses = {}
train_mse = None

for train_df, test_df in ds.iterate():
    X_train, y_train = ds.split_feature_label(train_df)
    
    if first_iter:
        catboost_hparams = {"depth": 5, "iterations": 250, "learning_rate": 0.03, "loss_function": "RMSE"}
        # Let's log the hyperparameters to W&B
        wandb.config.update(catboost_hparams)
        t = CatBoostTrainer(catboost_hparams)
        print(f"Training for {ds.get_partition_key(train_df)}...")
        # We'll pass in the W&B callback to log metrics
        t.fit(X_train, y_train, verbose=100, callbacks=[WandbCallback()])
        first_iter = False

        train_mse = t.score(X_train, y_train)
    
    # Evaluate
    X_test, y_test = ds.split_feature_label(test_df)
    test_mses[ds.get_partition_key(test_df)] = t.score(X_test, y_test)

In [ ]:
plt.plot(test_mses.keys(), [train_mse] * len(test_mses.keys()), label="train")
plt.plot(test_mses.keys(), test_mses.values(), label="test")
plt.xticks(rotation=45, ha='right')
plt.xlabel("Week")
plt.ylabel("MSE")
plt.title("MSE over week (no retraining)")
plt.legend()
# Let's log our plot to W&B so that we can refer to it in reports and dashboards
wandb.log({"MSE over week (no retraining)": wandb.Image(plt)})
plt.show()

Wow, it looks like the MSE gets significantly worse as we deploy over time! Makes sense, as the seasons change.

## Continual-Train

In [ ]:
test_mses = {}
train_mses = {}

for train_df, test_df in ds.iterate():
    X_train, y_train = ds.split_feature_label(train_df)
    
    catboost_hparams = {"depth": 5, "iterations": 250, "learning_rate": 0.03, "loss_function": "RMSE"}
    continual_t = CatBoostTrainer(catboost_hparams)
    continual_t.fit(X_train, y_train, verbose=False)
    train_mses[ds.get_partition_key(train_df)] = continual_t.score(X_train, y_train)
    
    # Evaluate
    X_test, y_test = ds.split_feature_label(test_df)
    test_mses[ds.get_partition_key(test_df)] = continual_t.score(X_test, y_test)

In [ ]:
plt.plot(train_mses.keys(), train_mses.values(), label="train")
plt.plot(test_mses.keys(), test_mses.values(), label="test")
plt.xticks(rotation=45, ha='right')
plt.xlabel("Week")
plt.ylabel("MSE")
plt.title("MSE over week (with retraining)")
plt.legend()
# Let's log our plot to W&B so that we can refer to it in reports and dashboards
wandb.log({"MSE over week (with retraining)": wandb.Image(plt)})
plt.show()

The MSEs improve here, because we're retraining on fresh data.

In [ ]:
continual_t.get_feature_importance().head(25)

In [ ]:
# We can finish now the W&B run
run.finish()